# Exploring Transformers

Exploring the hugging face transformers library through messing around with the documentation.

## Inference with pipelines

In [21]:
from transformers import pipeline

In [22]:
classifier = pipeline(model="facebook/bart-large-mnli")

In [23]:
classifier(
    "I have a problem with my iphone that needs to be resolved asap!!",
    candidate_labels=["urgent", "not urgent", "phone", "tablet", "computer"],
)

{'sequence': 'I have a problem with my iphone that needs to be resolved asap!!',
 'labels': ['urgent', 'phone', 'computer', 'not urgent', 'tablet'],
 'scores': [0.5036349892616272,
  0.4788002371788025,
  0.012600214220583439,
  0.0026557890232652426,
  0.002308773808181286]}

In [24]:
classifier(
    "There were three blue smurfs and a green dragon attacking a hobbit",
    candidate_labels=["fantasy", "history", "mathematics", "chemistry", "philosophy"],
)

{'sequence': 'There were three blue smurfs and a green dragon attacking a hobbit',
 'labels': ['fantasy', 'chemistry', 'philosophy', 'history', 'mathematics'],
 'scores': [0.955268919467926,
  0.013140400871634483,
  0.012286421842873096,
  0.010746616870164871,
  0.008557658642530441]}

In [25]:
import torch
from transformers import pipeline

pipe = pipeline(model="facebook/opt-1.3b", torch_dtype=torch.bfloat16, device_map="auto")
output = pipe("This is a cool example!", do_sample=True, top_p=0.95)

TypeError: BFloat16 is not supported on MPS

This error above is an issue with Apple Silicon, which is just an occupational hazard of an M-series chip.

In [26]:
import gradio as gr

pipe = pipeline("image-classification", model="google/vit-base-patch16-224")

gr.Interface.from_pipeline(pipe).launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


## Preprocess data

Load a pre-trained tokenizer for text data

In [27]:
from transformers import AutoTokenizer

In [28]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

In [29]:
encoded_input = tokenizer("Do not meddle in the affairs of wizards, for they are subtle and quick to anger.")
print(encoded_input)

{'input_ids': [101, 2091, 1136, 1143, 13002, 1107, 1103, 5707, 1104, 16678, 1116, 117, 1111, 1152, 1132, 11515, 1105, 3613, 1106, 4470, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


- `input_ids` are the indices corresponding to each token in the sentence.
- `attention_mask` indicates whether a token should be attended to or not.
- `token_type_ids` identifies which sequence a token belongs to when there is more than one sequence.

Can then also decode, to demonstrate the round-trip functionality.

In [30]:
tokenizer.decode(encoded_input["input_ids"])

'[CLS] Do not meddle in the affairs of wizards, for they are subtle and quick to anger. [SEP]'

the tokenizer added two special tokens - CLS and SEP (classifier and separator) - to the sentence. Not all models need special tokens, but if they do, the tokenizer automatically adds them for you.

This was great for one sentence, let's now tokenize more.

In [31]:
batch_sentences = [
    "But what about second breakfast?",
    "Don't think he knows about second breakfast, Pip.",
    "What about elevensies?",
]
encoded_inputs = tokenizer(batch_sentences)
print(encoded_inputs)

{'input_ids': [[101, 1252, 1184, 1164, 1248, 6462, 136, 102], [101, 1790, 112, 189, 1341, 1119, 3520, 1164, 1248, 6462, 117, 21902, 1643, 119, 102], [101, 1327, 1164, 5450, 23434, 136, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}


Again, follow up on round trip completeness.

In [34]:
[tokenizer.decode(input_sequence) for input_sequence in encoded_inputs["input_ids"]]

['[CLS] But what about second breakfast? [SEP]',
 "[CLS] Don't think he knows about second breakfast, Pip. [SEP]",
 '[CLS] What about elevensies? [SEP]']

That's all fine if we decode or do other processing one at a time, but transformer models need rectangular inputs to do operations on tensors. So, we need to pad to ensure all tokenized inputs have the same length. That's done with a flag, or alternately a token reserved for padding.

In [35]:
batch_sentences = [
    "But what about second breakfast?",
    "Don't think he knows about second breakfast, Pip.",
    "What about elevensies?",
]
encoded_input = tokenizer(batch_sentences, padding=True)
print(encoded_input)

{'input_ids': [[101, 1252, 1184, 1164, 1248, 6462, 136, 102, 0, 0, 0, 0, 0, 0, 0], [101, 1790, 112, 189, 1341, 1119, 3520, 1164, 1248, 6462, 117, 21902, 1643, 119, 102], [101, 1327, 1164, 5450, 23434, 136, 102, 0, 0, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]}


In [36]:
[tokenizer.decode(input_sequence) for input_sequence in encoded_input["input_ids"]]

['[CLS] But what about second breakfast? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 "[CLS] Don't think he knows about second breakfast, Pip. [SEP]",
 '[CLS] What about elevensies? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]']

And we can see the effect of the special padding in the decoded outputs.

Now, what if the reverse were true and the sequence of tokens is too long for the model? Truncation is the name of the game.

In [37]:
batch_sentences = [
    "But what about second breakfast?",
    "Don't think he knows about second breakfast, Pip.",
    "What about elevensies?",
]
encoded_input = tokenizer(batch_sentences, padding=True, truncation=True)
print(encoded_input)

{'input_ids': [[101, 1252, 1184, 1164, 1248, 6462, 136, 102, 0, 0, 0, 0, 0, 0, 0], [101, 1790, 112, 189, 1341, 1119, 3520, 1164, 1248, 6462, 117, 21902, 1643, 119, 102], [101, 1327, 1164, 5450, 23434, 136, 102, 0, 0, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]}


In [38]:
[tokenizer.decode(input_sequence) for input_sequence in encoded_input["input_ids"]]

['[CLS] But what about second breakfast? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 "[CLS] Don't think he knows about second breakfast, Pip. [SEP]",
 '[CLS] What about elevensies? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]']

While we've implemented truncation, looks like it doesn't have an effect here.

So now that we have the tokens on hand, we need to get this into a shape that can be accepted by the models, i.e. a tensor. Since I'm primarily a PyTorch user, we use the "pt" tensor parameter.

In [39]:
batch_sentences = [
    "But what about second breakfast?",
    "Don't think he knows about second breakfast, Pip.",
    "What about elevensies?",
]
encoded_input = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors="pt")
print(encoded_input)

{'input_ids': tensor([[  101,  1252,  1184,  1164,  1248,  6462,   136,   102,     0,     0,
             0,     0,     0,     0,     0],
        [  101,  1790,   112,   189,  1341,  1119,  3520,  1164,  1248,  6462,
           117, 21902,  1643,   119,   102],
        [  101,  1327,  1164,  5450, 23434,   136,   102,     0,     0,     0,
             0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


## Fine tune a pretrained model

Source: https://huggingface.co/docs/transformers/quicktour

In [1]:
from datasets import load_dataset

/Users/matt.heffernan/Code/venvs/base/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Get the dataset

In [2]:
dataset = load_dataset("yelp_review_full")

In [3]:
dataset["train"][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

Get the tokenizer

In [4]:
from transformers import AutoTokenizer

Let's use bert because this is just a demo, so we don't need anything fancy.

In [5]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

In [6]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [7]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Let's use small datsets so this doesn't take forever to run locally.

In [8]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

Train with PyTorch

In [9]:
from transformers import AutoModelForSequenceClassification

Actually fetch the pre-trained model. Because there are 5 stars in reviews, we know there are 5 labels (see the dataset documentation)

In [10]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Create class that'll contain hyperparameters from training as well as checkpoints.

In [11]:
from transformers import TrainingArguments

In [12]:
training_args = TrainingArguments(output_dir="test_trainer")

Evaluate model during training

In [15]:
import numpy as np
import evaluate
metric = evaluate.load("accuracy")

Compute the metrics converting logits to predictions.

In [16]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

specify the eval_strategy parameter in training arguments to report the evaluation metric at the end of each epoch

In [17]:
import torch
from transformers import TrainingArguments, Trainer

In [18]:
training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")


Now actually get the training off the ground. Initialize the trainer with all the setup we've done.

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

Now actually do the training.

In [20]:
trainer.train()

                                                 
 33%|███▎      | 125/375 [13:27<19:23,  4.65s/it]

{'eval_loss': 1.2962383031845093, 'eval_accuracy': 0.478, 'eval_runtime': 176.7811, 'eval_samples_per_second': 5.657, 'eval_steps_per_second': 0.707, 'epoch': 1.0}


                                                   
 67%|██████▋   | 250/375 [26:53<10:14,  4.92s/it]

{'eval_loss': 0.9933936595916748, 'eval_accuracy': 0.575, 'eval_runtime': 174.6948, 'eval_samples_per_second': 5.724, 'eval_steps_per_second': 0.716, 'epoch': 2.0}


                                                   
100%|██████████| 375/375 [40:14<00:00,  6.44s/it]

{'eval_loss': 1.0068022012710571, 'eval_accuracy': 0.588, 'eval_runtime': 173.3425, 'eval_samples_per_second': 5.769, 'eval_steps_per_second': 0.721, 'epoch': 3.0}
{'train_runtime': 2414.6244, 'train_samples_per_second': 1.242, 'train_steps_per_second': 0.155, 'train_loss': 1.098946533203125, 'epoch': 3.0}


TrainOutput(global_step=375, training_loss=1.098946533203125, metrics={'train_runtime': 2414.6244, 'train_samples_per_second': 1.242, 'train_steps_per_second': 0.155, 'total_flos': 789354427392000.0, 'train_loss': 1.098946533203125, 'epoch': 3.0})

## Distributed training with accelerate

Fast, convenient distribution of training over multiple compute resources.

In [40]:
from accelerate import Accelerator

accelerator = Accelerator()

Prepare to accelerate by passing the relevant (i.e. all) components to the accelerator.

In [ ]:
train_dataloader, eval_dataloader, model, optimizer = accelerator.prepare(
    train_dataloader, eval_dataloader, model, optimizer
)

Then replace loss.backward with accelerate's own method in the training loop.

In [ ]:
for epoch in range(num_epochs):
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

This is really just demonstration pseudocode without having defined the detailed components. Because this is now reaching the point where specifics are needed, the exploratory notebook is best finished and more details should be put in scripts and/or more specific explorations.